Tutorial 2 - Basic Flowsheet Optimization
=============================

Introduction
------------

In the previous tutorial, we developed a model for a simple flowsheet to simulate the performance of a series of reactions occurring in two CSTRs in series. In this tutorial, we will move from simulating to optimizing the flowsheet by trying to improve the yield of the main product (Sodium Acetate) but determining the optimal volumes for the two tanks subject to a limitation on the total volume of the reactors.

<img src="2cstrs.png">

<center>ethyl acetate + NaOH $\rightarrow$ sodium acetate + ethanol</center>

In this tutorial, you will learn how to:

* add an objective function to a model,
* set bounds on variables,
* add degrees of freedom to the model, and
* solve the resulting optimization problem.

A completed example of this tutorial as a Python script is available in the same folder as this Notebook.

First Steps
-----------

For this tutorial we are going to build on the model used in the last tutorial. Rather than rebuild the entire model, we will instead import the pre-built script in the same folder as this Notebook and run it.

In [ ]:
import Tutorial_1_Basic_Flowsheets
m, r = Tutorial_1_Basic_Flowsheets.main()

This step should have loaded and solved the model from Tutorial 1, and printed the results above. It should also have returned the entire model object (named `m`) so that we can continue to work with it.

In the last tutorial, we only imported those packages and components we need to solve a fully determined problem. In this tutorial however, we are going to perform an optimization, thus we need to import some additional components from Pyomo to set up the objective function.

* `Constraint` will be used to add a limit to the total volume of the reactors in our process,
* `Objective` will be used to specify the objective function for the problem, and
* `maximize` will be used to specify that the `Objective` should be maximized.
* `SolverFactory` needs to be imported again, so we can use it in this notebook.

To import these components, we use the following:

```py
from pyomo.environ import Constraint, Objective, maximize, SolverFactory
```

Adding an Objective Function
----------------------------

Now that we have an initialized flowsheet for our problem, we can go about adding an objective function. For this tutorial, we ant to maximize to yield of our desired component, Sodium Acetate.

For this, we add an `Objective` object to our flowsheet (`m.fs` as you may recall) and provide it with an expression for the objective function (in this case the concentration of component Sodium Acetate leaving `Tank2`) and an instruction on whether to minimize and maximize this expression.

```py
m.fs.obj = Objective(
            expr=m.fs.Tank2.outlet.conc_mol_comp[0, "SodiumAcetate"],
            sense=maximize)
```

Next, we want to add a constraint to the maximum volume of the two reactors (otherwise the problem will be unbounded and the tanks will go to infinite volume). For our constraint, let us set the total volume of the reactors to be equal to 3 $m^3$ (we could use an inequality constraint here, but we know that production will always increase with volume, so it will always hit this bound).

```py
m.fs.volume_constraint = Constraint(expr=m.fs.Tank1.volume[0] + m.fs.Tank2.volume[0] == 3.0)
```

Adding Variable Bounds
----------------------

Next, we need to add some limits on our problem to make sure the results of the optimization are physically reasonable. Our property packages have already set some limits on the state variables in the problem (such as a minimum value of 0 for concentrations), but let us add some limits on the volumes of each tank, such that volume must be between 0.5 and 5 $m^3$ (although we know the combined volume of the tanks must be 3, lets set the bound above this).

We can set bound by using the `setlb` (lower bound) and `setub` (upper bound) attributes of the volume variables.

```py
m.fs.Tank1.volume[0].setlb(0.5)
m.fs.Tank1.volume[0].setub(5.0)
m.fs.Tank2.volume[0].setlb(0.5)
m.fs.Tank2.volume[0].setub(5.0)
```

Finally, we need to allow the solver to change the values of the volumes of each tank. To do this, we `unfix` both these variables as shown:

```py
m.fs.Tank1.volume.unfix()
m.fs.Tank2.volume.unfix()
```

Solving the Optimization
------------------------------------

Now that our model is fully setup, we need to setup a solver again and apply it to our model. The code to do so is shown below:


```py
solver = SolverFactory('ipopt')
results = solver.solve(m, tee=True)
```

Viewing Results
------------------------

Let us first check the results object returned by the solver to see our degrees of freedom.

```py
print(results)
```

You should see 41 constraints and 42 variables in the problem, and that an optimal solution was found.

Next, let us look at the volumes of the two tanks to see what solution the solver found.

```py
print("Tank 1 Volume")
m.fs.Tank1.volume.display()
print()
print("Tank 2 Volume")
m.fs.Tank2.volume.display()
```

You should see the following results:

Tank 1 volume = 1.215 m^3

Tank 2 volume = 1.785 m^3

These volumes sum to 3 m^3 as expected, and show that the solver is suggesting that `Tank2` should be larger than `Tank1`. This makes sense as the concentration of reactants, and hence reaction rates, are higher in `Tank1`, whilst `Tank2` is operating at lower concentrations to polish off the remaining reactants (and hence needs a larger volume).

Finally, let us look at the outlet conditions from the two tanks to see how this affected production.

```py
print("Tank 1 Outlet")
m.fs.Tank1.outlet.display()
print()
print("Tank 2 Outlet")
m.fs.Tank2.outlet.display()
```

If all has gone well, you should see the following results.

**Tank 1**
* flow_vol = 1.0 [m^3/s]
* conc_mol_comp["SodiumAcetate"] = 77.689 [mol/m^3]
* conc_mol_comp["Ethanol"] = 77.689 [mol/m^3]
* conc_mol_comp["NaOH"] = 22.311 [mol/m^3]
* conc_mol_comp["EthylAcetate"] = 22.311 [mol/m^3]
* conc_mol_comp["H2O"] = 55388.0 [mol/m^3]
* pressure = 101325 [Pa]
* temperature = 304.06 [K]

**Tank 2**
* flow_vol = 1.0 [m^3/s]
* conc_mol_comp["SodiumAcetate"] = 92.106 [mol/m^3]
* conc_mol_comp["Ethanol"] = 92.106 [mol/m^3]
* conc_mol_comp["NaOH"] = 7.894 [mol/m^3]
* conc_mol_comp["EthylAcetate"] = 7.894 [mol/m^3]
* conc_mol_comp["H2O"] = 55388.0 [mol/m^3]
* pressure = 101325 [Pa]
* temperature = 303.15 [K]

Compared to the previous tutorial, we can see that conversion in `Tank1` has increased slightly (roughly 2%) due to the increase in volume in `Tank1`. With the additional increase in volume in `Tank2`, our overall conversion has increased by over 2% as well, from 89.6% in the unoptimized case to 92.1% in the optimized case.